## Download necessary files

In [1]:
# Clone GFPGAN and enter the GFPGAN folder
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN


!pip install basicsr --quiet
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan --quiet

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models


Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 523 (delta 168), reused 153 (delta 153), pack-reused 313
Receiving objects: 100% (523/523), 5.37 MiB | 8.49 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/GFPGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... do

## Take input video

In [15]:
import os
import shutil
import cv2
from google.colab import files

upload_folder = "video/inputs"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)

# Upload video
uploaded = files.upload()

for filename in uploaded.keys():
    video_path = os.path.join(upload_folder, filename)
    print(f'Moving {filename} to {video_path}')
    shutil.move(filename, video_path)

    # Extract frames from the video
    video_capture = cv2.VideoCapture(video_path)
    frame_rate = int(video_capture.get(5))  # Get frames per second (fps)

    frame_folder = os.path.join(upload_folder, os.path.splitext(filename)[0])
    os.makedirs(frame_folder, exist_ok=True)

    frame_count = 0
    while True:
        success, frame = video_capture.read()
        if not success:
            break

        # Save frame
        frame_filename = f"{frame_count:04d}.jpg"
        frame_path = os.path.join(frame_folder, frame_filename)

        if not os.path.exists(frame_path):
            cv2.imwrite(frame_path, frame)

        frame_count += 1  # Capture every frame

    video_capture.release()
    print(f'Frames saved to {frame_folder}')


Saving test1.mp4 to test1.mp4
Moving test1.mp4 to video/inputs/test1.mp4
Frames saved to video/inputs/test1


## Start upscaling and noise removal


In [11]:
input_folder = frame_folder
print(f"Input frames are taken from: {input_folder}")
output_folder = os.path.join('video/output/images/', os.path.splitext(filename)[0])
print(f"output frames are saved to: {output_folder}")

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder, exist_ok=True)

Input frames are taken from: video/inputs/test1
output frames are saved to: video/output/images/test1


In [16]:
output_folder = os.path.join('video/output/images/', os.path.splitext(filename)[0])
output_folder

'video/output/images/test1'

## Start Procesing

In [14]:
input_folder = frame_folder
print(f"Input frames are taken from: {input_folder}")
output_folder = os.path.join('video/output/images/', os.path.splitext(filename)[0])
print(f"output frames are saved to: {output_folder}")

Input frames are taken from: video/inputs/test1
output frames are saved to: video/output/images/test1


In [17]:
# Run the inference command
!python inference_gfpgan.py -i {input_folder} -o {output_folder} -v 1.3 -s 2 --bg_upsampler realesrgan


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing 0000.jpg ...
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Processing 0001.jpg ...
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/

## Display and Download output

In [26]:
file_name = os.path.splitext(filename)[0]
output_folder = f'video/output/images/{file_name}/restored_imgs'
output_video_path = f'video/output/{filename}'
output_folder

'video/output/images/test1/restored_imgs'

In [20]:
import cv2
import os
from IPython.display import HTML
from base64 import b64encode

output_folder = f'video/output/images/{file_name}/restored_imgs'
output_video_path = f'video/output/{filename}'

# Get the list of frames
frames = [os.path.join(output_folder, frame) for frame in os.listdir(output_folder) if frame.endswith('.jpg')]

# Sort frames based on their names
frames.sort()

# Read the first frame to get dimensions
frame = cv2.imread(frames[0])
height, width, _ = frame.shape

# Create a video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can try other codecs like 'XVID' based on your system support
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

# Write frames to video
for frame_path in frames:
    img = cv2.imread(frame_path)
    out.write(img)

out.release()

In [21]:
# Display the video in Colab
video_encoded = b64encode(open(output_video_path, 'rb').read()).decode('utf-8')
video_tag = f'<video controls alt="video" src="data:video/mp4;base64,{video_encoded}" width="{width}" height="{height}">'
HTML(video_tag)

In [22]:
from google.colab import files
files.download(output_video_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>